In [4]:
# import sys
# !conda install --yes --prefix {sys.prefix} tqdm
# !conda install --yes --prefix {sys.prefix} tensorflow

Solving environment: ...working... done

## Package Plan ##

  environment location: F:\Anaconda3

  added / updated specs:
    - tqdm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.1.23  |                0         158 KB
    certifi-2019.3.9           |           py36_0         156 KB
    cryptography-2.6.1         |   py36h7a1dbc1_0         563 KB
    kiwisolver-1.0.1           |   py36h6538335_0          61 KB
    krb5-1.16.1                |       hc04afaa_7         819 KB
    libcurl-7.64.0             |       h2a8f88b_2         283 KB
    libpng-1.6.36              |       h2a8f88b_0         550 KB
    openssl-1.1.1b             |       he774522_1         5.7 MB
    pycurl-7.43.0.2            |   py36h7a1dbc1_0         182 KB
    pyqt-5.9.2                 |   py36h6538335_2         4.2 MB
    qt-5.9.7                   |   vc14h73c81de_0        92.3 MB
    

Solving environment: ...working... done

# All requested packages already installed.



In [12]:
import time, sys
from IPython.display import clear_output
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
bar_length  = 50
block = int(round(bar_length * progress))
clear_output(wait = True)
text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
print(text)

NameError: name 'bar_length' is not defined

In [1]:
import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install ipdb
# !{sys.executable} -m pip install tensorflow_hub
!{sys.executable} -m pip install --upgrade tensorflow 

Requirement already up-to-date: tensorflow in f:\anaconda3\lib\site-packages (1.13.1)


In [9]:
!{sys.executable} -m pip install tensorflow-gpu 

Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'F:\\Anaconda3\\Lib\\site-packages\\tensorflow\\python\\_pywrap_tensorflow_internal.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
from tqdm import tqdm

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import urllib
import sys
import os
import zipfile

Download Glove word embeddings

In [4]:
glove_zip_file = "data/glove.6B.zip"
glove_vectors_file = "data/glove.6B.50d.txt"
import zipfile, urllib.request, shutil, os
    
#large file - 862 MB
if (not os.path.isfile(glove_zip_file) and
    not os.path.isfile(glove_vectors_file)):
    with urllib.request.urlopen("http://nlp.stanford.edu/data/glove.6B.zip") as response, open(glove_zip_file, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

Unzip word embeddings

In [5]:
def unzip_single_file(zip_file_name, output_file_name):
    """
        If the outFile is already created, don't recreate
        If the outFile does not exist, create it from the zipFile
    """
    if not os.path.isfile(output_file_name):
        with open(output_file_name, 'wb') as out_file:
            with zipfile.ZipFile(zip_file_name) as zipped:
                for info in zipped.infolist():
                    if output_file_name in info.filename:
                        with zipped.open(info) as requested_file:
                            out_file.write(requested_file.read())
                            return

unzip_single_file(glove_zip_file, glove_vectors_file)

In [6]:
glove_wordmap = {}
with open(glove_vectors_file, "r", encoding="utf8") as glove:
    for line in glove:
        name, vector = tuple(line.split(" ", 1))
        glove_wordmap[name] = np.fromstring(vector, sep=" ")

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

def sentence2sequence(sentence):
    """
     
    - Turns an input sentence into an (n,d) matrix, 
        where n is the number of tokens in the sentence
        and d is the number of dimensions each word vector has.
    
      Tensorflow doesn't need to be used here, as simply
      turning the sentence into a sequence based off our 
      mapping does not need the computational power that
      Tensorflow provides. Normal Python suffices for this task.
    """
    # GLOVE
#     tokens = sentence.lower().split(" ")
#     rows = []
#     words = []
#     #Greedy search for tokens
#     for token in tokens:
#         i = len(token)
#         while len(token) > 0 and i > 0:
#             word = token[:i]
#             if word in glove_wordmap:
#                 rows.append(glove_wordmap[word])
#                 words.append(word)
#                 token = token[i:]
#                 i = len(token)
#             else:
#                 i = i-1
#     return rows, words

# ELMO

#     with tf.Session() as sess:
    emb = elmo(sentence,
    signature="default",
    as_dict=True)["elmo"]
#         sess.run(tf.global_variables_initializer())
#         embeddings = sess.run(emb)
    return embeddings
print(sentence2sequence(["apple carrots"]))

[[[ 0.7557078  -0.29991025  0.74354523 ...,  0.26082098  0.32645333
    0.48831999]
  [ 0.33039808  0.22212878  0.71501005 ...,  0.39743879  0.19633918
    0.45981079]]]


In [2]:
#Constants setup
max_hypothesis_length, max_evidence_length = 30, 30
batch_size, vector_size, hidden_size = 128, 50, 64

lstm_size = hidden_size

weight_decay = 0.0001

learning_rate = 1

input_p, output_p = 0.5, 0.5

training_iterations_count = 100000

display_step = 10

def score_setup(row):
    convert_dict = {
      'ENTAILMENT': 0,
      'NEUTRAL': 1,
      'CONTRADICTION': 2
    }
    score = np.zeros((3,))
    tag = row["entailment_judgment"]
    score[convert_dict[tag]] += 1
    return score

def fit_to_size(matrix, shape):
    res = np.zeros(shape)
    slices = [slice(0,min(dim,shape[e])) for e, dim in enumerate(matrix.shape)]
    res[slices] = matrix[slices]
    return res

### [Training data](http://www.site.uottawa.ca/~diana/csi5386/A2_2019/SICK_train.txt)

In [61]:
hyp_sentences_embedded = "data/elmo_hyp_sent_emb_%s.txt"
evi_sentences_embedded = "data/elmo_evi_sent_emb_%s.txt"
import os, pickle
def save_elmo_emb(hyp_sentences, evi_sentences, stage=""):    
    global hyp_sentences_embedded
    global evi_sentences_embedded
    hyp_file = hyp_sentences_embedded % stage
    evi_file = evi_sentences_embedded % stage
    if not os.path.isfile(hyp_file):
        print("writing %s" % hyp_file)
        with open(hyp_file, "wb") as f:
#             pickler = pickle.Pickler(f)
            pickle.dump(hyp_sentences, f)
    if not os.path.isfile(evi_file):
        print("writing %s" % evi_file)
        with open(evi_file, "wb") as f:
#             pickler = pickle.Pickler(f)
            pickle.dump(evi_sentences, f)

In [60]:
import os
def load_elmo_emb_hyp():
    if os.path.isfile(hyp_sentences_embedded):
        with open(hyp_sentences_embedded, "rb") as f:
#             unpickler = pickle.Unpickler(f)
#             hyp_sentences = unpickler.load()
            hyp_sentences = pickle.load()
            return hyp_sentences
def load_elmo_emb_evi():
    if os.path.isfile(evi_sentences_embedded):
        with open(evi_sentences_embedded, "rb") as f:
#             unpickler = pickle.Unpickler(f)
#             evi_sentences = unpickler.load()
            evi_sentences = pickle.load()

            return evi_sentences
    

In [8]:
import numpy as np
import time, datetime
import tensorflow as tf

def embed_elmo(train, start_stage=0):
    started = time.time()
    last_stage = time.time()

    hyp_sentences, evi_sentences = [], []
    count = 0
    number_of_elements = 10000
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for row in train:
            count += 1
            if count <= start_stage:
                continue
            if (count-start_stage) % 10 == 0:
                print("Stage %s. Time elapsed so far: %s" % (str(count), str(time.time()-started)))
                stage = str(count)
                save_elmo_emb(hyp_sentences, evi_sentences, stage)
                hyp_sentences, evi_sentences = [], []

            print("Stage %s. Time elapsed last stage: %s" % (str(count), str(time.time()-last_stage)))
            last_stage = time.time()

            hyp_sentences.append(#np.vstack(
                    sess.run(sentence2sequence([row["sentence_A"].lower()[0]])))
    #             passed = time.time()
    #             print("Emb %s S1: %s" % (str(count), str(passed-started)))
            evi_sentences.append(#np.vstack(
                    sess.run(sentence2sequence([row["sentence_B"].lower()[0]])))
    #             started = time.time()
    #             print("Emb %s S2: %s" % (str(count), str(started-passed)))
    #         update_progress(count / number_of_elements)
    print("Total exe time: %s" % (str(time.time()-started)))
    return hyp_sentences, evi_sentences, labels, scores

def split_data_into_scores():
    print("started")
    import csv
    with open("data/training.txt","r") as data:
        print("opened training file")
        train = csv.DictReader(data , delimiter='\t')
        evi_sentences = load_elmo_emb_hyp()
        hyp_sentences = load_elmo_emb_evi()
        if evi_sentences is None or hyp_sentences is None:
            hyp_sentences, evi_sentences = embed_elmo(train)
        hyp_sentences = np.stack([fit_to_size(x, (max_hypothesis_length, vector_size))
                          for x in hyp_sentences])
        evi_sentences = np.stack([fit_to_size(x, (max_evidence_length, vector_size))
                          for x in evi_sentences])
        save_elmo_emb(hyp, evi)
        
        labels = []
        scores = []
        for row in train:
            if count % 10==0:
                print("Stage %s. Time elapsed so far: %s" % (str(count), str(time.time()-started)))
            count+=1
            labels.append(row["entailment_judgment"])
            scores.append(score_setup(row))
        
        print("Split took: %s" %(str(datetime.timedelta(seconds=time.time()-started))))    

        return (hyp_sentences, evi_sentences), labels, np.array(scores)
update_progress(1)
data_feature_list, correct_values, correct_scores = split_data_into_scores()

l_h, l_e = max_hypothesis_length, max_evidence_length
N, D, H = batch_size, vector_size, hidden_size
l_seq = l_h + l_e

NameError: name 'update_progress' is not defined

In [ ]:
tf.reset_default_graph()

In [ ]:
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
lstm_drop =  tf.contrib.rnn.DropoutWrapper(lstm, input_p, output_p)

In [ ]:
# N: The number of elements in each of our batches, 
#   which we use to train subsets of data for efficiency's sake.
# l_h: The maximum length of a hypothesis, or the second sentence.  This is
#   used because training an RNN is extraordinarily difficult without 
#   rolling it out to a fixed length.
# l_e: The maximum length of evidence, the first sentence.  This is used
#   because training an RNN is extraordinarily difficult without 
#   rolling it out to a fixed length.
# D: The size of our used GloVe or other vectors.
hyp = tf.placeholder(tf.float32, [N, l_h, D], 'hypothesis')
evi = tf.placeholder(tf.float32, [N, l_e, D], 'evidence')
y = tf.placeholder(tf.float32, [N, 3], 'label')
# hyp: Where the hypotheses will be stored during training.
# evi: Where the evidences will be stored during training.
# y: Where correct scores will be stored during training.

# lstm_size: the size of the gates in the LSTM, 
#    as in the first LSTM layer's initialization.
lstm_back = tf.contrib.rnn.BasicLSTMCell(lstm_size)
# lstm_back:  The LSTM used for looking backwards 
#   through the sentences, similar to lstm.

# input_p: the probability that inputs to the LSTM will be retained at each
#   iteration of dropout.
# output_p: the probability that outputs from the LSTM will be retained at 
#   each iteration of dropout.
lstm_drop_back = tf.contrib.rnn.DropoutWrapper(lstm_back, input_p, output_p)
# lstm_drop_back:  A dropout wrapper for lstm_back, like lstm_drop.


fc_initializer = tf.random_normal_initializer(stddev=0.1) 
# fc_initializer: initial values for the fully connected layer's weights.
# hidden_size: the size of the outputs from each lstm layer.  
#   Multiplied by 2 to account for the two LSTMs.
fc_weight = tf.get_variable('fc_weight', [2*hidden_size, 3], 
                            initializer = fc_initializer)
# fc_weight: Storage for the fully connected layer's weights.
fc_bias = tf.get_variable('bias', [3])
# fc_bias: Storage for the fully connected layer's bias.

# tf.GraphKeys.REGULARIZATION_LOSSES:  A key to a collection in the graph
#   designated for losses due to regularization.
#   In this case, this portion of loss is regularization on the weights
#   for the fully connected layer.
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, 
                     tf.nn.l2_loss(fc_weight)) 

x = tf.concat([hyp, evi], 1) # N, (Lh+Le), d
# Permuting batch_size and n_steps
x = tf.transpose(x, [1, 0, 2]) # (Le+Lh), N, d
# Reshaping to (n_steps*batch_size, n_input)
x = tf.reshape(x, [-1, vector_size]) # (Le+Lh)*N, d
# Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
x = tf.split(x, l_seq,)

# x: the inputs to the bidirectional_rnn


# tf.contrib.rnn.static_bidirectional_rnn: Runs the input through
#   two recurrent networks, one that runs the inputs forward and one
#   that runs the inputs in reversed order, combining the outputs.
rnn_outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm, lstm_back,
                                                            x, dtype=tf.float32)
# rnn_outputs: the list of LSTM outputs, as a list. 
#   What we want is the latest output, rnn_outputs[-1]

classification_scores = tf.matmul(rnn_outputs[-1], fc_weight) + fc_bias
# The scores are relative certainties for how likely the output matches
#   a certain entailment: 
#     0: Positive entailment
#     1: Neutral entailment
#     2: Negative entailment

In [14]:
with tf.variable_scope('Accuracy'):
    predicts = tf.cast(tf.argmax(classification_scores, 1), 'int32')
    y_label = tf.cast(tf.argmax(y, 1), 'int32')
    corrects = tf.equal(predicts, y_label)
    num_corrects = tf.reduce_sum(tf.cast(corrects, tf.float32))
    accuracy = tf.reduce_mean(tf.cast(corrects, tf.float32))

with tf.variable_scope("loss"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
        logits = classification_scores, labels = y)
    loss = tf.reduce_mean(cross_entropy)
    total_loss = loss + weight_decay * tf.add_n(
        tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

opt_op = optimizer.minimize(total_loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [15]:
# Initialize variables
init = tf.global_variables_initializer()

# Use TQDM if installed
tqdm_installed = False
try:
    from tqdm import tqdm
    tqdm_installed = True
except:
    pass

# Launch the Tensorflow session
sess = tf.Session()
sess.run(init)

# training_iterations_count: The number of data pieces to train on in total
# batch_size: The number of data pieces per batch
training_iterations = range(0,training_iterations_count,batch_size)
if tqdm_installed:
    # Add a progress bar if TQDM is installed
    training_iterations = tqdm(training_iterations)

for i in training_iterations:
    if i % 1000 == 0:
    # Select indices for a random data subset
        batch = np.random.randint(data_feature_list[0].shape[0], size=batch_size)
    
    # Use the selected subset indices to initialize the graph's 
    #   placeholder values
    hyps, evis, ys = (data_feature_list[0][batch,:],
                      data_feature_list[1][batch,:],
                      correct_scores[batch])
    
    # Run the optimization with these initialized values
    sess.run([opt_op], feed_dict={hyp: hyps, evi: evis, y: ys})
    # display_step: how often the accuracy and loss should 
    #   be tested and displayed.
    if (i/batch_size) % display_step == 0:
        # Calculate batch accuracy
        acc = sess.run(accuracy, feed_dict={hyp: hyps, evi: evis, y: ys})
        # Calculate batch loss
        tmp_loss = sess.run(loss, feed_dict={hyp: hyps, evi: evis, y: ys})
        # Display results
        print("Iter " + str(i/batch_size) + ", Minibatch Loss= " + \
              "{:.6f}".format(tmp_loss) + ", Training Accuracy= " + \
              "{:.5f}".format(acc))

  0%|                                                                                                                                                                                   | 0/782 [00:00<?, ?it/s]

Iter 0.0, Minibatch Loss= 1.088545, Training Accuracy= 0.54688


  1%|█▉                                                                                                                                                                         | 9/782 [00:03<18:19,  1.42s/it]

Iter 10.0, Minibatch Loss= 0.955285, Training Accuracy= 0.54688


  2%|███▋                                                                                                                                                                      | 17/782 [00:03<09:04,  1.40it/s]

...

 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉   | 768/782 [00:25<00:00, 34.15it/s]

Iter 770.0, Minibatch Loss= 0.828612, Training Accuracy= 0.64844


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 780/782 [00:25<00:00, 33.41it/s]

Iter 780.0, Minibatch Loss= 0.708883, Training Accuracy= 0.66406


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 782/782 [00:25<00:00, 30.39it/s]


In [17]:
evidences = ["People wearing costumes are gathering in a forest and are looking in the same direction"]
hypotheses = ["Masked people are looking in the same direction in a forest"
]



sentence1 = [fit_to_size(np.vstack(sentence2sequence(evidence)[0]),
                         (30, 50)) for evidence in evidences]

sentence2 = [fit_to_size(np.vstack(sentence2sequence(hypothesis)[0]),
                         (30,50)) for hypothesis in hypotheses]


import numpy as np

def split_data_into_scores():
    import csv
    with open("data/test_labeled.txt","r") as data:
        train = csv.DictReader(data , delimiter='\t')
        evi_sentences = []
        hyp_sentences = []
        labels = []
        scores = []
        count = 1
        for row in train:
            hyp_sentences.append(np.vstack(
                    sentence2sequence(row["sentence_A"].lower())[0]))
            evi_sentences.append(np.vstack(
                    sentence2sequence(row["sentence_B"].lower())[0]))
            labels.append(row["entailment_judgment"])
            scores.append(score_setup(row))
        
        hyp_sentences = np.stack([fit_to_size(x, (max_hypothesis_length, vector_size))
                          for x in hyp_sentences])
        evi_sentences = np.stack([fit_to_size(x, (max_evidence_length, vector_size))
                          for x in evi_sentences])
                                 
        return (hyp_sentences, evi_sentences), labels, np.array(scores)
data_feature_list, correct_values, correct_scores = split_data_into_scores()




hyps, evis, ys = (data_feature_list[0][batch,:],
                  data_feature_list[1][batch,:],
                  correct_scores[batch])
predictions = sess.run(classification_scores, feed_dict={hyp: hyps, evi: evis, y: ys})

acc = sess.run(accuracy, feed_dict={hyp: hyps, evi: evis, y: ys})
print(acc)

total = len(predictions)
correct_predictions = 0
for i,prediction in enumerate(predictions):
    if np.argmax(prediction[0])==np.argmax(ys[i]):
        correct_predictions += 1
#     print(["Positive", "Neutral", "Negative"][np.argmax(prediction[0])]+
#       " / " + ["Positive", "Neutral", "Negative"][np.argmax(ys[i])])
    
print("Acc: %s" % str(correct_predictions*100/total))

Positive / Positive
Positive / Neutral
Positive / Negative
Positive / Positive
Positive / Neutral
Positive / Neutral
Positive / Neutral
Positive / Positive
Positive / Neutral
Positive / Neutral
Positive / Neutral
Positive / Negative
Positive / Neutral
Positive / Neutral
Positive / Neutral
Positive / Negative
Positive / Negative
Positive / Positive
Positive / Positive
Positive / Neutral
Positive / Neutral
Positive / Negative
Positive / Negative
Positive / Positive
Positive / Positive
Positive / Neutral
Positive / Neutral
Positive / Positive
Positive / Neutral
Positive / Neutral
Positive / Neutral
Positive / Positive
Positive / Positive
Positive / Positive
Positive / Negative
Positive / Neutral
Positive / Positive
Positive / Neutral
Positive / Neutral
Positive / Negative
Positive / Positive
Positive / Neutral
Positive / Neutral
Positive / Positive
Positive / Positive
Positive / Neutral
Positive / Negative
Positive / Neutral
Positive / Neutral
Positive / Positive
Positive / Positive
Posit

In [ ]:
sess.close()

In [22]:
# def visualize(sentence):
#     rows, words = sentence2sequence(sentence)
#     mat = np.vstack(rows)
    
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     shown = ax.matshow(mat, aspect="auto")
#     ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
#     fig.colorbar(shown)
    
#     ax.set_yticklabels([""]+words)
#     plt.show()
    
# visualize("The quick brown fox jumped over the lazy dog.")
# visualize("The pretty flowers shone in the sunlight.")

TypeError: Can't convert 'text': Shape TensorShape([]) is incompatible with TensorShape([Dimension(None)])